In [43]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
client = OpenAI()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
reviews_df = pd.read_csv('datasets/full-reviews.csv')

# Send the prompt to ChatGPT
def chat_gpt_summarize(review):
    response = client.chat.completions.create(
    model="gpt-4o",
    response_format={ "type": "text" },
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to summarize reviews and return a short **summary**."},
    {"role": "user", "content": f"Summarize this review: '{review}"},
  ]
)
    return response.choices[0].message.content

def chat_gpt_sentiment(review):
    response = client.chat.completions.create(
    model="gpt-4o",
    response_format={ "type": "text" },
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to read reviews and return a short **sentiment analysis**."},
    {"role": "user", "content": f"Give me a short sentiment analysis of this review: '{review}"},
  ]
)
    return response.choices[0].message.content

def chat_gpt_score(review):
    response = client.chat.completions.create(
    model="gpt-4o",
    response_format={ "type": "text" },
    messages=[
    {"role": "system", "content": "You are a rating assistant designed to read reviews and return a single number score between 0 & 100. 100 is for an extremely positive review and 0 is for an extremely negative review."},
    {"role": "user", "content": f"Please rate this review on a scale from 0-100 based on how much they enjoyed the movie. Return only a single number with no explanation please: '{review}"},
  ]
)
    return response.choices[0].message.content


In [13]:
specific_movie_reviews = reviews_df[reviews_df['movie_title'] == "Percy Jackson & the Olympians: The Lightning Thief"]

In [14]:
specific_movie_reviews.head()

,movie_title,review_content,review_type
0,Percy Jackson & the Olympians: The Lightning T...,A fantasy adventure that fuses Greek mythology...,Fresh
1,Percy Jackson & the Olympians: The Lightning T...,"Uma Thurman as Medusa, the gorgon with a coiff...",Fresh
2,Percy Jackson & the Olympians: The Lightning T...,With a top-notch cast and dazzling special eff...,Fresh
3,Percy Jackson & the Olympians: The Lightning T...,Whether audiences will get behind The Lightnin...,Fresh
4,Percy Jackson & the Olympians: The Lightning T...,What's really lacking in The Lightning Thief i...,Rotten


In [16]:
specific_movie_reviews.shape

(149, 3)

In [19]:
reviews_df['movie_title'].value_counts(ascending=False)

movie_title
Star Wars: The Rise of Skywalker        992
Solo: A Star Wars Story                 948
Star Wars: The Last Jedi                946
Rogue One: A Star Wars Story            892
Spider-Man: Far From Home               880
                                       ... 
The Influence (La influencia)             5
Thick as Thieves (The Code)               5
Adventures of a Teenage Dragonslayer      4
You Get Me                                4
Farewell                                  1
Name: count, Length: 17100, dtype: int64

In [50]:
summaries = {}
sentiments = {}

In [51]:
for review in specific_movie_reviews['review_content']:
    summaries[review] = chat_gpt_summarize(review)
    sentiments[chat_gpt_sentiment(review)] = chat_gpt_score(review)


In [52]:
print(summaries)

{'A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle': 'This fantasy adventure blends Greek mythology with modern American settings and values, creating a visually thrilling experience that will particularly appeal to teens around the age of 15.', 'Uma Thurman as Medusa, the gorgon with a coiffure of writhing snakes and stone-inducing hypnotic gaze is one of the highlights of this bewitching fantasy': 'Uma Thurman’s portrayal of Medusa, with her snake hair and petrifying gaze, is a standout element in this captivating fantasy.', 'With a top-notch cast and dazzling special effects, this will tide the teens over until the next Harry Potter instalment.': 'The review praises the film for its excellent cast and impressive special effects, suggesting it will entertain teens until the next Harry Potter release.', "Whether audiences will get behind The Lightning Thief i

In [53]:
print(sentiments)

{'The review is positive, highlighting the thrilling visual spectacle of the fantasy adventure and its unique fusion of Greek mythology with contemporary American elements. It particularly emphasizes appeal for teenagers around 15 years old.': 'Based on the review provided, I would rate it an 80. The reviewer appreciates the blend of Greek mythology with contemporary American culture and highlights that it will be thrilling, especially for a younger audience.', 'Positive sentiment. The review praises Uma Thurman\'s performance as Medusa and describes the movie as a "bewitching fantasy," highlighting her role as one of its notable features.': 'Based on the positive language used to describe Uma Thurman\'s performance and the movie itself as "bewitching," I would rate this review an 85 out of 100.', 'The sentiment of the review is positive. The reviewer praises the cast and special effects, suggesting the film is enjoyable and will keep teens entertained.': 'Based on the review, I would 

In [64]:
#convert the dictionary to a dataframe
summaries_df = pd.DataFrame(summaries.items(), columns=['review', 'summary'])
sentiments_df = pd.DataFrame(sentiments.items(), columns=['sentiment', 'score'])

In [65]:
#delete all non-numeric values from the score column
sentiments_df = sentiments_df[sentiments_df['score'].str.isnumeric()]

In [74]:
sentiments_df.reset_index(drop=True, inplace=True)
sentiments_df.head(20)

,sentiment,score
0,The review presents a mixed but generally posi...,70
1,The sentiment of the review is positive. The r...,80
2,The sentiment of the review is negative. The r...,20
3,The sentiment of the review is negative. The r...,20
4,The sentiment of the review is mildly positive...,45
5,"The sentiment of the review is positive, highl...",80


In [70]:
summaries_df.head()

,review,summary
0,A fantasy adventure that fuses Greek mythology...,This fantasy adventure blends Greek mythology ...
1,"Uma Thurman as Medusa, the gorgon with a coiff...","Uma Thurman’s portrayal of Medusa, with her sn..."
2,With a top-notch cast and dazzling special eff...,The review praises the film for its excellent ...
3,Whether audiences will get behind The Lightnin...,"The review suggests that ""The Lightning Thief""..."
4,What's really lacking in The Lightning Thief i...,"The review criticizes ""The Lightning Thief"" fo..."


In [72]:
#combine the summaries and sentiments dataframes
combined_df = pd.concat([summaries_df, sentiments_df], axis=1)

In [73]:
combined_df.head(20)

,review,summary,sentiment,score
0,A fantasy adventure that fuses Greek mythology...,This fantasy adventure blends Greek mythology ...,The review presents a mixed but generally posi...,70
1,"Uma Thurman as Medusa, the gorgon with a coiff...","Uma Thurman’s portrayal of Medusa, with her sn...",The sentiment of the review is positive. The r...,80
2,With a top-notch cast and dazzling special eff...,The review praises the film for its excellent ...,The sentiment of the review is negative. The r...,20
3,Whether audiences will get behind The Lightnin...,"The review suggests that ""The Lightning Thief""...",The sentiment of the review is negative. The r...,20
4,What's really lacking in The Lightning Thief i...,"The review criticizes ""The Lightning Thief"" fo...",The sentiment of the review is mildly positive...,45
5,It's more a list of ingredients than a movie-m...,The review suggests that the film feels disjoi...,"The sentiment of the review is positive, highl...",80
6,Harry Potter knockoffs don't come more transpa...,This review criticizes the movie as a poorly m...,NaN,NaN
7,"Percy Jackson isn't a great movie, but it's a ...","""Percy Jackson"" isn't a great movie, but it is...",NaN,NaN
8,"Fun, brisk and imaginative","The review describes the experience as fun, br...",NaN,NaN
9,"Crammed with dragons, set-destroying fights an...",This review describes a action-packed film fil...,NaN,NaN
